In [1]:
import warnings 
warnings.filterwarnings('ignore')
from sklearn.datasets import load_iris
import numpy as np
import math
from tqdm import tqdm
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
import xgboost as xgb
import category_encoders.utils as util
from category_encoders import TargetEncoder
from sklearn.feature_extraction.text import CountVectorizer

Lectura de archivos

In [2]:
iden = pd.read_csv('train_identity.csv')

In [3]:
trans = pd.read_csv('train_transaction.csv')

Merge de df

In [4]:
joined = trans.merge(iden,how='left')

In [5]:
del(joined['TransactionID'])

Libero RAM

In [6]:
del iden

In [7]:
del trans

Busco el valor limite entre train y validation para hacer el split

In [8]:
limite = np.percentile(joined['TransactionDT'], 80)

Genero el DF de Validation

In [9]:
filtro_validation = joined['TransactionDT'] > limite

In [10]:
validation = joined[filtro_validation]

In [11]:
label_validation = validation[['isFraud']]

In [12]:
del(validation['isFraud'])

Genero el DF de train

In [13]:
filtro_train = joined['TransactionDT']<= limite

In [14]:
train = joined[filtro_train]

In [15]:
label_train = train[['isFraud']]

In [16]:
del(train['isFraud'])

Libero RAM


In [17]:
del joined

### Encodings

Creo un diccionario para almancenar los encodings.

In [18]:
dict_encoding = {}

Creo funciones para aplicar los encodings.

In [19]:
def countVect(df,columna,dic):
    if columna not in dic:
        dic[columna] = CountVectorizer(max_features =10)
        col_encoded = dic[columna].fit_transform(df[columna].fillna('desconocido')).astype(int).toarray()
    else:
        col_encoded = dic[columna].transform(df[columna].fillna('desconocido')).astype(int).toarray()
    col_encoded = pd.DataFrame(col_encoded).add_prefix(columna+ '_enconded_').fillna(0)
    del(df[columna])
    df_nuevo = pd.concat([df.reset_index(drop=True),col_encoded], axis=1)
    return df_nuevo

In [20]:
def one_hot_encode(df, columna, dict_encoding):
    if columna not in dict_encoding:
        dict_encoding[columna] = OneHotEncoder(handle_unknown='ignore')
        dict_encoding[columna].fit_transform(df[[columna]].astype(str)).astype(int)
    col_encoded = (dict_encoding[columna].transform(df[[columna]].astype(str)).astype(int))
    col_encoded = pd.DataFrame(col_encoded.todense()).add_prefix(columna + '_').fillna(0)
    del(df[columna])
    df_nuevo = pd.concat([df.reset_index(drop=True), col_encoded], axis=1)
    return df_nuevo

In [21]:
def mean_encoding_train(df,columna,label, dict_encoding):
    if columna not in dict_encoding:
        dict_encoding[columna] = TargetEncoder(handle_unknown='ignore')
    df[columna] = dict_encoding[columna].fit_transform(df[[columna]], label)
    df[columna].fillna(0,inplace=True)
    return df

In [22]:
def mean_encoding_val_test(df,columna, dict_encoding):
    df[columna] = dict_encoding[columna].transform(df[[columna]])
    df[columna].fillna(0,inplace=True)
    return df

In [23]:
def label_ec(df):
    for column in df.columns:
        if df[column].dtypes==object:
            dict_encoding[column] = OrdinalEncoder(handle_unknown='ignore')
            df[column] = dict_encoding[column].fit_transform(df[column].astype(str).values.reshape(-1, 1))
    if df[column].dtype==object:
        df[column] = dict_encoding[column].transform(df[column].astype(str).values.reshape(-1, 1))
    df.fillna(-1,inplace=True)
    return(df)

Defino los features que les aplicare los encodings: oneHot, mean o CountVectorizer

In [24]:
countV = ['id_30','id_31','DeviceInfo']

In [25]:
mean = ['addr2','DeviceType']

In [26]:
oneHot = ['P_emaildomain','card4','ProductCD','id_38','id_35','id_34']

### Modelo

In [27]:
xboost = xgb.XGBClassifier(random_state=0)

In [28]:
#xboost.get_xgb_params()

### Train

Realizo los encodings.

In [29]:
del(train['TransactionDT'])

In [30]:
del(train['TransactionAmt'])

In [31]:
for i in countV:
    train=countVect(train,i,dict_encoding)

In [32]:
for i in oneHot:
    train = one_hot_encode(train,i,dict_encoding)

In [33]:
for i in mean:
    train = mean_encoding_train(train,i,label_train,dict_encoding)

In [34]:
train = label_ec(train)

Aplico la tecnica de feature selection

In [35]:
from sklearn.feature_selection import SelectFromModel

embeded_lgb_selector = SelectFromModel(xboost, max_features=len(train.columns))
embeded_lgb_selector.fit(train, label_train)

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = train.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

[13:24:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
125 selected features


Me quedo con los features seleccionados.

In [36]:
train = train [embeded_lgb_feature]

Entreno el modelo

In [37]:
xboost.fit(train,label_train)

[13:28:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Validation


Realizo los encodings.

In [38]:
for i in countV:
    validation=countVect(validation,i,dict_encoding)

In [39]:
for i in oneHot:
    validation = one_hot_encode(validation,i,dict_encoding)

In [40]:
for i in mean:
    validation = mean_encoding_val_test(validation,i,dict_encoding)

In [41]:
validation = label_ec(validation)

Me quedo con los features seleccionados.

In [42]:
validation = validation[embeded_lgb_feature]

Se muestra el classification report.

In [43]:
print(classification_report(label_validation, xboost.predict(validation)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99    114044
           1       0.71      0.37      0.49      4064

    accuracy                           0.97    118108
   macro avg       0.84      0.68      0.74    118108
weighted avg       0.97      0.97      0.97    118108



Calculo el ***roc auc*** del validation.

In [44]:
roc_auc_score(label_validation, xboost.predict_proba(validation)[:,1])

0.8795445683935501